# **Feature Selection and Data Classification Assignment**

# **Step 1: Preprocessing**

This step prepares the dataset for model training by handling outliers, missing values, encoding, scaling, and feature selection.

Data Loading
* The dataset is loaded from Google Drive for analysis.

Outlier Detection
* We identify and remove outliers in numerical columns using Z-score filtering.
* Data points with a Z-score greater than 3 are removed to reduce noise and improve model performance.

Handling Missing Values
* Missing values are imputed using the mean for numerical columns, ensuring we retain all rows without introducing gaps in the data.

Encoding Categorical Variables
* Categorical columns (e.g., `Race`, `Marital Status`) are one-hot encoded, converting them into a numerical format that machine learning models can process.

Mapping the Target Variable
* The target variable, `Status`, is mapped to binary values (`0` for "Alive" and `1` for "Dead") for classification.

Data Splitting
* We split the dataset into training and testing sets with a 70-30 split to train and evaluate the model effectively.

Standardization
* Features are standardized using `StandardScaler` to ensure uniform scaling, which is crucial for distance-based models like KNN.

Feature Selection
* Using Information Gain, we calculate feature importance scores to identify the top features that contribute most to predicting survivability.


In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from google.colab import drive
drive.mount('/content/drive')

# Access the file from Google Drive
data = pd.read_csv('/content/drive/My Drive/PA_HW3/Breast_Cancer_dataset.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
from scipy import stats

# Remove any leading/trailing spaces from column names
data.columns = data.columns.str.strip()

# Outlier Detection using Z-Score
z_scores = np.abs(stats.zscore(data.select_dtypes(include=[np.number])))
data_no_outliers = data[(z_scores < 3).all(axis=1)]

# Separating target variable
y = data["Status"].copy()
X = data.drop(columns=["Status"])

# Detect categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
X = pd.get_dummies(X, columns=categorical_columns)

if y.dtype == 'object':
    y = y.map({"Alive": 0, "Dead": 1})

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# Impute missing values in X_train and X_test
imputer = SimpleImputer(strategy="mean")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Standardize the features after imputing missing values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature Selection using Information Gain
info_gain = mutual_info_classif(X_train_scaled, y_train)
feature_importances = pd.Series(info_gain, index=X.columns).sort_values(ascending=False)
top_features = feature_importances[:10].index
print("Top Features based on Information Gain:\n", feature_importances.head(10))



Top Features based on Information Gain:
 Survival Months                 0.137639
Progesterone Status_Positive    0.027779
6th Stage_IIIC                  0.024776
Reginol Node Positive           0.024068
N Stage_N1                      0.020010
N Stage_N3                      0.017142
Progesterone Status_Negative    0.014711
Tumor Size                      0.014541
6th Stage_IIA                   0.013018
Grade_3                         0.011348
dtype: float64


**Dimensionality Reduction with PCA**

To simplify the dataset and reduce potential noise, we apply **Principal Component Analysis (PCA)** to transform the features into a lower-dimensional space.

* **Purpose**: PCA helps to reduce the number of features while retaining the most important information, which can speed up model training and reduce the risk of overfitting.

* **Number of Components**: We choose the top 10 principal components, capturing the majority of the variance in the data.

The transformed datasets (`X_train_pca` and `X_test_pca`) are used for model training and testing, providing a more compact representation of the original data.


In [51]:
# Using PCA to reduce dimensions
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


# **Step 2: Modeling**

In this step, we train several machine learning algorithms on the preprocessed dataset. Each model has unique strengths and weaknesses, making them suitable for different types of data and tasks. We also implement feature selection techniques, using the most informative features to improve model performance.

### 1. K-Nearest Neighbors (KNN)
* **Description**: KNN is a distance-based algorithm that classifies data points based on the majority label of their k nearest neighbors.
* **Implementation**: We implement KNN from scratch without using any built-in libraries for the classification.
* **Pros**: Simple and interpretable; works well for small datasets.
* **Cons**: Computationally expensive for large datasets and sensitive to feature scaling.
* **Main Hyperparameter**: `k` (number of neighbors).

### 2. Naive Bayes
* **Description**: A probabilistic classifier based on Bayes' theorem, assuming independence between features.
* **Pros**: Fast and efficient; performs well with high-dimensional data.
* **Cons**: Assumes feature independence, which may not hold in all datasets, potentially reducing accuracy.
* **Main Hyperparameter**: `var_smoothing` (for Gaussian Naive Bayes).

### 3. Decision Tree (C4.5)
* **Description**: A tree-based model that splits data into branches based on feature values, making decisions at each node.
* **Pros**: Interpretable and flexible, works with both numerical and categorical data.
* **Cons**: Prone to overfitting, especially with deep trees.
* **Main Hyperparameter**: `max_depth` (controls the depth of the tree), `criterion` (split criteria, e.g., Gini or Entropy).

### 4. Random Forest
* **Description**: An ensemble of decision trees that combines predictions from multiple trees to improve accuracy and reduce overfitting.
* **Pros**: Robust to overfitting; can handle high-dimensional data and provides feature importance insights.
* **Cons**: Less interpretable than a single decision tree; computationally intensive with many trees.
* **Main Hyperparameters**: `n_estimators` (number of trees), `max_depth` (depth of each tree).

### 5. Gradient Boosting
* **Description**: An ensemble method that builds trees sequentially, with each tree correcting the errors of the previous ones.
* **Pros**: High accuracy and suitable for imbalanced data.
* **Cons**: Computationally intensive; prone to overfitting if not carefully tuned.
* **Main Hyperparameters**: `n_estimators` (number of boosting stages), `learning_rate` (controls the contribution of each tree).

### 6. Neural Network (Multi-layer Perceptron)
* **Description**: A multi-layer perceptron that learns complex patterns through layers of interconnected neurons.
* **Pros**: Can capture complex, non-linear relationships in the data.
* **Cons**: Computationally expensive; requires more data and is prone to overfitting if not regularized.
* **Main Hyperparameters**: `hidden_layer_sizes` (number of neurons in each layer), `activation` (activation function), `max_iter` (maximum number of training iterations).


In [52]:
import numpy as np

class SimpleKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for row in X:
            distances = np.linalg.norm(self.X_train - row, axis=1)
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y_train[k_indices]
            predictions.append(np.bincount(k_nearest_labels).argmax())
        return predictions

knn = SimpleKNN(k=3)
knn.fit(X_train_scaled, y_train)

predictions = {}
predictions["KNN"] = knn.predict(X_test_scaled)


In [53]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

nb = GaussianNB()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
nn = MLPClassifier()

models = {
    "Naive Bayes": nb,
    "Decision Tree": dt,
    "Random Forest": rf,
    "Gradient Boosting": gb,
    "Neural Network": nn
}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    predictions[name] = model.predict(X_test_scaled)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# **Step 3: Hyperparameter Tuning**

In this step, we perform hyperparameter tuning on two of the models: **Random Forest** and **Gradient Boosting**. By using **Grid Search** with cross-validation, we explore different combinations of hyperparameters to identify the optimal settings for each model. This process helps to improve model performance by finding the best configuration of parameters.

### 1. Random Forest Hyperparameter Tuning
* **Parameters Tuned**:
  - `n_estimators`: Number of trees in the forest, tested values are 50 and 100.
  - `max_depth`: Maximum depth of each tree, tested values are `None` (unrestricted depth) and 10.

### 2. Gradient Boosting Hyperparameter Tuning
* **Parameters Tuned**:
  - `n_estimators`: Number of boosting stages, tested values are 50 and 100.
  - `learning_rate`: Step size for each stage, tested values are 0.1 and 0.01.


In [55]:
from sklearn.model_selection import GridSearchCV

tuned_params = {
    "Random Forest": {'n_estimators': [50, 100], 'max_depth': [None, 10]},
    "Gradient Boosting": {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.01]}
}

best_params = {}
for name, params in tuned_params.items():
    search = GridSearchCV(models[name], params, cv=3)
    search.fit(X_train_scaled, y_train)
    best_params[name] = search.best_params_

for model, params in best_params.items():
    print(f"Best parameters for {model}: {params}")

Best parameters for Random Forest: {'max_depth': 10, 'n_estimators': 100}
Best parameters for Gradient Boosting: {'learning_rate': 0.1, 'n_estimators': 100}


# **Step 4: Results**

In this step, we evaluate the performance of each model using several metrics: **Accuracy**, **Precision**, **Recall**, and **F1 Score**. These metrics provide a comprehensive view of each model's effectiveness in predicting breast cancer survivability. Additionally, for models that allow feature importance evaluation, we identify the most important features contributing to the classification.

### Model Performance

The table below summarizes the performance of each model:

| Model             | Accuracy | Precision | Recall | F1 Score |
|-------------------|----------|-----------|--------|----------|
| KNN               | 0.836093 | 0.396396  | 0.251429 | 0.307692 |
| Naive Bayes       | 0.816225 | 0.385366  | 0.451429 | 0.415789 |
| Decision Tree     | 0.852649 | 0.492386  | 0.554286 | 0.521505 |
| Random Forest     | 0.906457 | 0.810000  | 0.462857 | 0.589091 |
| Gradient Boosting | 0.912252 | 0.805310  | 0.520000 | 0.631944 |
| Neural Network    | 0.888245 | 0.651515  | 0.491429 | 0.560261 |

### Top Features by Importance (Random Forest)

Using the Random Forest model, we identify the top 10 features based on their importance scores:

| Feature                      | Importance |
|------------------------------|------------|
| Survival Months              | 0.337096   |
| Age                          | 0.111125   |
| Regional Node Examined       | 0.094744   |
| Tumor Size                   | 0.091198   |
| Reginol Node Positive        | 0.073965   |
| Marital Status_Married       | 0.021563   |
| Race_White                   | 0.016464   |
| 6th Stage_IIIC               | 0.014079   |
| Progesterone Status_Negative | 0.013580   |
| Estrogen Status_Negative     | 0.013064   |

### Conclusion

The objective of this project was to predict breast cancer survivability. Based on the results, **Gradient Boosting** and **Random Forest** achieved the highest accuracy and F1 scores, indicating strong predictive power for this task. These models are well-suited for capturing complex patterns in the data, making them effective for this classification problem.

The most informative features (e.g., **Survival Months**, **Age**, **Regional Node Examined**) align with clinical expectations, suggesting that the model insights are consistent with real-world factors that impact survivability.

In summary, we were able to answer the initial question of predicting breast cancer survivability effectively, especially with the use of ensemble models.


In [56]:
results = {}
for name, preds in predictions.items():
    results[name] = {
        "Accuracy": accuracy_score(y_test, preds),
        "Precision": precision_score(y_test, preds),
        "Recall": recall_score(y_test, preds),
        "F1 Score": f1_score(y_test, preds)
    }

results_df = pd.DataFrame(results).T

# Display Feature Importance for Random Forest
if "Random Forest" in models:
    feature_importance = pd.Series(models["Random Forest"].feature_importances_, index=X.columns).sort_values(ascending=False)
    print("Top Features by Importance in Random Forest:\n", feature_importance.head(10))

print("Model Performance:\n", results_df)

Top Features by Importance in Random Forest:
 Survival Months                 0.337096
Age                             0.111125
Regional Node Examined          0.094744
Tumor Size                      0.091198
Reginol Node Positive           0.073965
Marital Status_Married          0.021563
Race_White                      0.016464
6th Stage_IIIC                  0.014079
Progesterone Status_Negative    0.013580
Estrogen Status_Negative        0.013064
dtype: float64
Model Performance:
                    Accuracy  Precision    Recall  F1 Score
KNN                0.836093   0.396396  0.251429  0.307692
Naive Bayes        0.816225   0.385366  0.451429  0.415789
Decision Tree      0.852649   0.492386  0.554286  0.521505
Random Forest      0.906457   0.810000  0.462857  0.589091
Gradient Boosting  0.912252   0.805310  0.520000  0.631944
Neural Network     0.888245   0.651515  0.491429  0.560261
